--- 
### 3-13장 Chest X-Ray images 2. Baseline Model  
- https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia 
- 경진대회가 아닌 이미지셋만 주어짐 
- X-ray 이미지로 부터 정상 또는 폐렴 여부를 판별하는 문제 
- train / val /test 폴더에 각각 Normal/Pneumonia 폴더가 있다. (정상/폐렴의 이미지)
- 평가지표로 정확도, 재현율, F1-score 사용 
- Model: efficientnet-b0, Optimizer: Adam 사용 
---

In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

file_count = 0  # 출력한 파일 개수를 추적
max_files = 10  # 출력할 파일의 최대 개수

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        file_count += 1
        if file_count >= max_files:
            break
    if file_count >= max_files:
        break

/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1952_bacteria_4883.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1951_bacteria_4882.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/.DS_Store


In [4]:
# 1. 시드고정 
import torch 
import random 
import numpy as np 
import pandas as pd 
import os

seed = 50 
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

# 2.GPU setting 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

- 주어진 데이터는 train/valid set이 폴더단위로 구분되어 있다. 
- 따라서 data split와 Dataset class 정의가 불필요 --> ImageFolder로 간단하게 데이터셋 생성가능  

In [6]:
# 3. 데이터 로딩 
data_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/'
train_path = os.path.join(data_path, 'train/')
val_path = os.path.join(data_path, 'val/')
test_path = os.path.join(data_path, 'test/')

# 4. 이미지 증강을 위한 데이터 변환기 정의 
from torchvision import transforms 

transform_train = transforms.Compose([transforms.Resize((250,250)),   # 이미지 크기를 250x250으로 
                                      transforms.CenterCrop(180),     # 이미지 가운데 부분을 180x180 만큼 잘라냄 
                                      transforms.RandomHorizontalFlip(0.5), # 50% 확률로 수평변환 
                                      transforms.RandomVerticalFlip(0.2),   # 20% 확률로 수직변환 
                                      transforms.RandomRotation(20),        # +/- 20도 랜덤 회전 
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # RGB 평균&표준편차로 표준화 
                                      ])

transform_test = transforms.Compose([transforms.Resize((250,250)), 
                                      transforms.CenterCrop(180),                                       
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                      ])

# 5. 데이터셋 생성 
# ImageFolder는 root의 경로에 있는 이미지들을 바로 데이터셋으로 만들어준다. 
from torchvision.datasets import ImageFolder  

dataset_train = ImageFolder(root=train_path, transform=transform_train)
dataset_valid = ImageFolder(root=test_path, transform=transform_test)

# 6. 멀티프로세싱 설정, 제너레이터 시드고정 
def seed_worker(worker_id): 
    worker_seed = torch.initial_seed() % 2**32
    random.seed(worker_seed)
    np.random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

# 7. 데이터로더 생성 
from torch.utils.data import DataLoader
batch_size = 8
loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g, num_workers=2 )
loader_valid = DataLoader(dataset=dataset_valid, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2 )

In [8]:
# 8. efficientnet 모델 임포트 및 디바이스 할당 
!pip install efficientnet-pytorch==0.7.1
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2 )  # efficientnet-b0 : 약 4백만개의 parameters 
model = model.to(device)

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16425 sha256=ba6f62144b0a779a8f99dd53cba23564ae96fb7163424b26d97b2d7a34971e60
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 101MB/s] 


Loaded pretrained weights for efficientnet-b0


In [9]:
# 9. 손실함수, 옵티마이저 정의 
import torch.nn as nn 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
# 10. 훈련, 검증, 최적 가중치 찾기를 위함 함수 개발 
# 에폭 수만큼 훈련과 검증을 반복하면서 최적 모델의 가중치를 찾아서 마지막에 반환하는 구조다.
# 데이터 로더를 이용하므로 훈련과 검증을 미니배치 단위로 수행한다.

from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score   # 재현율 계산 함수
from sklearn.metrics import f1_score       # F1 점수 계산 함수
from tqdm.notebook import tqdm             # 진행률 표시 막대

def train(model, loader_train, loader_valid, criterion, optimizer, 
          scheduler=None, epochs=10, save_file='model_state_dict.pth'):
    
    valid_loss_min = np.inf # 최소 손실값 초기화 (검증 데이터용) 

    # 총 에폭만큼 반복
    for epoch in range(epochs):
        print(f'에폭 [{epoch+1}/{epochs}] \n-----------------------------')
        
        # == [ 훈련 ] ==============================================
        model.train()        # 모델을 훈련 상태로 설정
        epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
        # '반복 횟수'만큼 반복 
        for images, labels in tqdm(loader_train):
            # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당 
            images = images.to(device)
            labels = labels.to(device)
            
            # 옵티마이저 내 기울기 초기화
            optimizer.zero_grad()
            # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
            outputs = model(images)
            # 손실 함수를 활용해 outputs와 labels의 손실값 계산
            loss = criterion(outputs, labels)
            # 현재 배치에서의 손실 추가 (훈련 데이터용)
            epoch_train_loss += loss.item() 
            loss.backward()       # 역전파 수행
            optimizer.step()      # 가중치 갱신
            if scheduler != None: # 스케줄러 학습률 갱신 
                scheduler.step() 

        # 훈련 데이터 손실값 출력
        print(f'\t훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
        
        # == [ 검증 ] ==============================================
        model.eval()         # 모델을 평가 상태로 설정 
        epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용)
        preds_list = []      # 예측값 저장용 리스트 초기화
        true_list = []       # 실젯값 저장용 리스트 초기화
        
        with torch.no_grad(): # 기울기 계산 비활성화
            for images, labels in loader_valid:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                
                # 예측값 및 실제값 
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy() 
                true = labels.cpu().numpy() 
    
                preds_list.extend(preds)
                true_list.extend(true)
                
        # 정확도, 재현율, F1 점수 계산
        val_accuracy = accuracy_score(true_list, preds_list)
        val_recall = recall_score(true_list, preds_list)
        val_f1_score = f1_score(true_list, preds_list)

        # 검증 데이터 손실값 및 정확도, 재현율, F1점수 출력
        print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
        print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')
        # == [ 최적 모델 가중치 찾기 ] ==============================
        # 현 에폭에서의 손실값이 최소 손실값 이하면 모델 가중치 저장 
        if epoch_valid_loss <= valid_loss_min: 
            print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} --> {epoch_valid_loss:.4f}). 모델 저장')
            # 모델 가중치를 파일로 저장 
            torch.save(model.state_dict(), save_file) 
            valid_loss_min = epoch_valid_loss # 최소 손실값 갱신 
    return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환

In [11]:
# 11.train 함수로 훈련&검증 실행
model_state_dict = train(model=model, loader_train=loader_train, loader_valid=loader_valid, criterion=criterion, optimizer=optimizer)

에폭 [1/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.2076
	검증 데이터 손실값 : 0.4426
	정확도 : 0.8814 / 재현율 : 0.9897 / F1 점수 : 0.9125
	### 검증 데이터 손실값 감소 (inf --> 34.5198). 모델 저장
에폭 [2/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1220
	검증 데이터 손실값 : 0.4628
	정확도 : 0.8301 / 재현율 : 0.9872 / F1 점수 : 0.8790
에폭 [3/10] 
-----------------------------


...

  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0698
	검증 데이터 손실값 : 0.6472
	정확도 : 0.8317 / 재현율 : 0.9974 / F1 점수 : 0.8811
에폭 [10/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0691
	검증 데이터 손실값 : 0.3909
	정확도 : 0.8926 / 재현율 : 0.9821 / F1 점수 : 0.9196
	### 검증 데이터 손실값 감소 (30.5635 --> 30.4871). 모델 저장


<ipython-input-10-38d6a0076ed2>:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환


In [12]:
# 12.최적 가중치로 모델 업데이트 
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [13]:
# 13. 테스트 데이터로 예측실행 
# 테스트 데이터와 로더를 정의하고, 테스트용 함수 개발 
dataset_test = ImageFolder(root=test_path, transform=transform_test)
loader_test = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

def predict(model, loader_test, return_true=False): 
    model.eval() 
    preds_list=[]
    true_list=[]

    with torch.no_grad(): 
        for images, labels in loader_test: 
            images=images.to(device)
            labels=labels.to(device)
            outputs = model(images)

            preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
            preds_list.extend(preds)

            true = labels.cpu().numpy()
            true_list.extend(true)

    if return_true: 
        return true_list, preds_list
    else: 
        return preds_list

In [14]:
# 14.predict 함수로 예측 실행 
true_list, preds_list = predict(model=model, loader_test=loader_test, return_true=True)

In [16]:
# 15.평가지표 출력 
print('정확도 = ', accuracy_score(true_list, preds_list))
print('재현율 = ', recall_score(true_list, preds_list))
print('F1_socre = ', f1_score(true_list, preds_list))

정확도 =  0.8926282051282052
재현율 =  0.982051282051282
F1_socre =  0.9195678271308524
